In [189]:
import folium

In [190]:
kansas_city_coord = [39.085594, -94.585241]  # Kansas City, roughly the geographic center of the USA
san_fransisco_coord = [37.77, -122.41] #[37.77, 122.41]
boston_coord = [42.36, -71.06] #[42.36, 71.06]

# Create a blank map

In [191]:
def create_blank_map():
    m = folium.Map(location=kansas_city_coord, zoom_start=5, tiles='cartodbpositron', width='100%', height='100%')
    return m

In [192]:
create_blank_map()

# Basic markers, circles and lines
Marker with a colored icon, a tooltip and a multi-line popup. 

## Layer control
In order to be able to hide groups of elements, add them to a FeatureGroup.<br>
Then add a LayerControl to the map. This will automatically pickup any FeatureGroups.

## Icons
For ids of icons, see https://fontawesome.com/icons?d=gallery&q=shopp&m=free

## Multi-line popup
Work-around for multi-line in popup: https://github.com/python-visualization/folium/issues/469
`popup = (
        "Time: {time}<br>"
        "Speed: {speed} km/h<br>"
       ).format(time=row.name.strftime('%H:%M'), speed=str(round(row['spd'],2))`

In [193]:
def create_basic_markers_map():
    m = create_blank_map()
    fg = folium.FeatureGroup(name='Basic markers')
    
    # Marker with icon, tooltip and popup
    coord = boston_coord
    icon = folium.Icon(color='green',
        icon_color='white',
        icon='shopping-cart',
        angle=0, prefix='fa')
    popup = (
        "Hello, I am {}.<br>"
        "Welcome to my showcase in {}."
            ).format('Boston', 2019)
    tooltip = 'Boston'
    folium.Marker(coord,
        popup=popup,
        tooltip=tooltip,
        icon=icon
        ).add_to(fg)
    
    
    # Circle 1, tooltip and popup
    coord_1 = kansas_city_coord
    popup_1 = (
        "Hello, I am {}.<br>"
        "Welcome to my showcase in {}."
            ).format('Kansas', 2019)
    radius_1 = 20
    folium.CircleMarker(coord_1,
        radius=radius_1,
        popup=popup_1,
        tooltip='Kansas',
        color='blue',
        fill_color='green',
        fill=True,
        fill_opacity=0.4,
        stroke=True,
        ).add_to(fg)
    
    # Circle 2, tooltip and popup
    coord_2 = san_fransisco_coord
    popup_2 = (
        "Hello, I am {}.<br>"
        "Welcome to my showcase in {}."
            ).format('San Fransisco', 1972)
    radius_2 = 40
    folium.CircleMarker(coord_2,
        radius=radius_2,
        popup=popup_2,
        tooltip='San Fransisco',
        color='red',
        fill_color='orange',
        fill=True,
        fill_opacity=0.4,
        stroke=True,
        ).add_to(fg)
                 
    # Line
    coordinates = [kansas_city_coord, boston_coord]
    folium.PolyLine(coordinates, popup="Line", tooltip="Line tooltip", color='blue', weight=5).add_to(fg)
    

    fg.add_to(m)
    folium.LayerControl(position='topleft').add_to(m)
    return m

In [194]:
create_basic_markers_map()

# Arrow line

Thanks to Bob Haffner (https://medium.com/@bobhaffner/folium-lines-with-arrows-25a0fe88e4e)

In [195]:
def get_bearing(p1, p2):
    """
    Returns compass bearing from p1 to p2

    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon

    Return
    compass bearing of type float

    Notes
    Based on https://gist.github.com/jeromer/2005586
    """
    import numpy as np
    long_diff = np.radians(p2.lon - p1.lon)

    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)

    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2)
         - (np.sin(lat1) * np.cos(lat2)
            * np.cos(long_diff)))

    bearing = np.degrees(np.arctan2(x, y))

    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing

def get_arrows(locations, color='blue', size=6, n_arrows=3, add_to=None):
    """Add arrows to a hypothetical line between the first 2 locations in the locations list.
    Get a list of correctly placed and rotated arrows/markers to be plotted.

    Args:
        locations : List of lists of lat lon that represent the
                    start and end of the line.
                    eg [[41.1132, -96.1993],[41.3810, -95.8021]]
                    The locations is a list so that it matches the input for the folium.PolyLine.
        color : Whatever folium can use.  Default is 'blue'
        size : Size of arrow. Default is 6
        n_arrows : Number of arrows to create.  Default is 3.
        add_to: map or FeatureGroup the arrows are added to.

    Returns:
        list of arrows/markers
    """
    # TODO: generalize so that locations can be any length >=2, i.e. a PolyLine with more than 1 section.

    from collections import namedtuple
    import numpy as np
    Point = namedtuple('Point', field_names=['lat', 'lon'])

    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])

    # getting the rotation needed for our marker.
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90

    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows + 1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows + 1]

    arrows = []

    # creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points,
                                                  fill_color=color, number_of_sides=3,
                                                  radius=size, rotation=rotation).add_to(add_to))
    return arrows

In [196]:
def create_arrow_line_map():
    m = create_blank_map()
    fg = folium.FeatureGroup(name='Arrow line')
    
    # Line
    coordinates = [kansas_city_coord, boston_coord]
    folium.PolyLine(coordinates, popup="Line", tooltip="Line tooltip", color='blue', weight=5).add_to(fg)
    
    #Add arrows
    get_arrows(locations=coordinates, color='#FFFFFF', n_arrows=3, add_to=fg)
    
    fg.add_to(m)
    folium.LayerControl(position='topleft').add_to(m)
    return m

In [197]:
create_arrow_line_map()

# Popup Table
A multi-line popup is nice to be able to show more information, but often we need to show multiple properties of an item.<br>
In that case, formatting them in a table is much nicer.<br>
Folium can show HTML in a popup. We'll be generating a HTML table wit 2 columns. The below functions are generic and can display an arbritarily long table.<br>
Each row is a sequence with 2 text elements, one for each column.<br>
For a demo, see the next section.

In [198]:
def get_html_table(rows):
    """Creates 2 column html table for use in popups.
    Args:
        rows: List of sequences. Each sequence should have 2 string entries, one for each column

    Returns:
        html: a HTML formatted table of two columns
    """
    table_html_pattern = """<table style="width:100%">{}</table>"""
    # row_html_pattern = """<tr><td>{}:&nbsp</td><td>{}</td></tr>"""
    row_html_pattern = """<tr><td>{}&nbsp</td><td>{}</td></tr>"""
    rows_html = ''
    for row in rows:
        rows_html = rows_html + '\n' + row_html_pattern.format(row[0], row[1])
    html = table_html_pattern.format(rows_html)
    return html


def get_popup_table(rows):
    """Return a popup table to add as a popup/child to a folium element.

    Usage:
        popup_table = [
        ]
        popup = MapManager.get_popup_table(popup_table)
        Next, the popup object can be used in a popup argument of a Marker:
        marker = folium.Marker(coord,
                           popup=popup,
                           icon=icon
                           )
        Or added as a child:
        county.add_child(popup)

    Args:
        rows: List of sequences. Each sequence should have 2 string entries, one for each column.

    Returns:
        popup (folium.Popup)

    Notes
    Beware that a quote in the texts causes a problem (no map shows).
    This can be avoided by replacing the "\'" with something else.
    Unfortunately the option `parse_html=True` does not prevent the problem.
    Despite the suggestion in https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Popups.ipynb
    """
    html_text = get_html_table(rows)
    html = folium.Html(html_text, script=True)
    popup = folium.Popup(html, parse_html=True, max_width=2650)
    return popup

# Contours
Contours are typically represented by GeoJSON files.<br>
Contours can be added in 2 ways:
1. As a set, in one go. This is easy and fast. But can only represent one color and no popups or tooltips.
2. One-by-one. This allows for colored heatmaps and tooltips and popeps per geometry.

See also: https://github.com/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb

## Load the GeoJSON

In [199]:
import os
import json
import geopandas as gpd
root_dir = os.environ['DSX_PROJECT_DIR']
geo_json_file_name = 'US_Counties_2010_us_050_00_20m_NYState'
                      #US_Counties_2010_us_050_00_20m_NYState.geojson
geo_json_file_path = os.path.join(root_dir, 'datasets', geo_json_file_name + '.geojson')
GEO_JSON_DATA = json.load(open(geo_json_file_path))
GEO_JSON_DF = gpd.read_file(geo_json_file_path)

In [200]:
nys_center_coord = [43, -76]
def create_nys_blank_map():
    m = folium.Map(location=nys_center_coord, zoom_start=7, tiles='cartodbpositron', width='100%', height='100%')
    return m

In [201]:
def create_monochrome_contour_map():
    m = create_nys_blank_map()
    folium.GeoJson(GEO_JSON_DATA).add_to(m)
    return m
create_monochrome_contour_map()

In [202]:
GEO_JSON_DF.head(3)

,id,NAME,LSAD,CENSUSAREA,GEO_ID,COUNTY,STATE,geometry
0,446,Allegany,County,1029.308,0500000US36003,003,36,"POLYGON ((-78.20660599999999 41.999989, -78.30..."
1,447,Columbia,County,634.705,0500000US36021,021,36,"POLYGON ((-73.929626 42.078778, -73.921465 42...."
2,448,Genesee,County,492.936,0500000US36037,037,36,"POLYGON ((-78.464381 42.867461, -78.463887 42...."


Create a dataframe of counties. Simulates as if there more information by county from other sources.

In [203]:
counties = (GEO_JSON_DF[['NAME','CENSUSAREA']]
            .rename(columns={'NAME': 'location_id', 'CENSUSAREA': 'area'})
            .set_index(['location_id'], verify_integrity=True)
           )
counties['population'] = counties.area * 3
counties.head(3)

,area,population
location_id,,
Allegany,1029.308,3087.924
Columbia,634.705,1904.115
Genesee,492.936,1478.808


The core of the heatmap is done by looping over each row of the DataFrame that geopandas created from the geojson file.<br>
Each row is converted to json and used to create a folium.GeoJson object. <br>
The GeoJson constructor adds a style_function, which contains a callback function to get the fill color.<br>
The arguments of the callback can be specific properties of the json, which are retreived by: `feature['properties']['NAME']`.<br>
The callback function gets the county name, retreives the property `populatin` from another source and uses that in the colormap function to return the fill color for this contour.<br>
The colormap is a LinearColormap based on the minimum and maximum values of the population property.
The site http://colorbrewer2.org gives some good suggestions for color schemes for heatmaps.



In [204]:
import branca
def create_color_contour_map():
    m = create_nys_blank_map()
    fg = folium.FeatureGroup(name='Contours')
    
    geojson_df = GEO_JSON_DF
    
    #Color map:
    vmin = counties.population.min()
    vmax = counties.population.max()
    population_colormap = branca.colormap.LinearColormap(['red', '#ffc935'], vmin=vmin, vmax=vmax, caption='Population')
    population_colormap = population_colormap.to_step(10)
    population_colormap.caption = 'Population'
    population_colormap.add_to(m)
    
    #Fill color function:
    def get_fill_color(name):
        population = counties.at[name, 'population'] #geojson_df.query("id==@id").CENSUSAREA
        return population_colormap(population)
    
    
    for i in range(geojson_df.shape[0]):
        county = folium.GeoJson(
            geojson_df.take([i], axis=0).to_json(),
            name='geojson',
            tooltip=folium.features.GeoJsonTooltip(fields=['NAME'], labels=False),
            style_function=lambda feature: {
                'fillColor': get_fill_color(feature['properties']['NAME']),
                'color': 'black',
                'fillOpacity': 0.6,
#                 'tooltip': 'Hello', #Doesn't work
                'weight': 2,
            }
        ).add_to(fg)
        


        row1 = geojson_df.iloc[i]
        location_id = row1.NAME
        county_row = counties.loc[location_id]
        popup_table = [('County', location_id),
                       ('Population', '{:.0f}'.format(county_row.population)),
                       ('Area', '{:.1f} sqm'.format(county_row.area)),
                       ('Assigned Trucks', '{}'.format(10)),
                       ('Utilization', '{:.0%}'.format(0.33)),
                       ('Relative Backlog', '{:.2%}'.format(0.02)),
                       ]
        popup = get_popup_table(popup_table)
        county.add_child(popup)

    fg.add_to(m)
    folium.LayerControl(position='topleft').add_to(m)
    return m
create_color_contour_map()

# Plotly graph in popup

In [205]:
import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)   # for offline mode use

Generic function that creates an iframe from a Plotly figure

In [206]:
def get_plotly_div(fig):
    """Get a folium.IFrame from a plotly.Figure
    """
    fig_div = py.offline.plot(fig, include_plotlyjs=False, output_type='div')
    # Adding the custom script section works fine
    # However, also tried without and then enabling the include_plotlyjs=True, causes the folium map generation to stall/hang
    html="""
        <script src='https://cdn.plot.ly/plotly-latest.min.js'></script>
        {}
        """.format(fig_div)
#     iframe = folium.IFrame(html=html, width=500, height=300)
    return html

In [207]:
def get_plotly_iframe(fig):
    """Get a folium.IFrame from a plotly.Figure
    """
    fig_div = py.offline.plot(fig, include_plotlyjs=False, output_type='div')
    # Adding the custom script section works fine
    # However, also tried without and then enabling the include_plotlyjs=True, causes the folium map generation to stall/hang
    html="""
        <script src='https://cdn.plot.ly/plotly-latest.min.js'></script>
        {}
        """.format(fig_div)
    iframe = folium.IFrame(html=html, width=500, height=300)
    return iframe

Example of a function that creates a Plotly pie chart based on a DataFrame

In [208]:
def get_plotly_pie_chart(df):
    layout = go.Layout(
        title='<b>Revenue by brand</b>',
        yaxis=dict(
            title='<i>Revenue</i>'
        ),
        xaxis=dict(
            title='<i>Brand</i>'
        )
    )

    data=[]
    trace1 = go.Pie(
            labels=df.brand,
            values=df.market_share,
        )
    data.append(trace1)

    fig = go.Figure(data=data, layout=layout)
    return fig

Some sample data for the pie chart

In [209]:
import pandas as pd
MY_DATA = pd.DataFrame([
    {'brand':'Costco', 'market_share':'0.40'},
    {'brand':'Walmart', 'market_share':'0.30'},
    {'brand':'Apple', 'market_share':'0.20'},
    {'brand':'Tesla', 'market_share':'0.10'},
])

Test the Ploy chart itself

In [210]:
fig = get_plotly_pie_chart(MY_DATA)
py.offline.iplot(fig)

Adds the Ployly chart as a popup in folium.

In [211]:
def create_plotly_popup_map():
    m = create_blank_map()
    fg = folium.FeatureGroup(name='Basic marker')
    
    fig = get_plotly_pie_chart(MY_DATA)
    iframe = get_plotly_iframe(fig)
    popup = folium.Popup(iframe, max_width=2650)
    
    # Marker with icon, tooltip and popup
    coord = boston_coord
    icon = folium.Icon(color='green',
        icon_color='white',
        icon='shopping-cart',
        angle=0, prefix='fa')
    tooltip = 'Boston'
    folium.Marker(coord,
        popup=popup,
        tooltip=tooltip,
        icon=icon
        ).add_to(fg)
    
    fg.add_to(m)
    folium.LayerControl(position='topleft').add_to(m)
    return m
create_plotly_popup_map()

# Experimental
Combine table and Potly chart in one HTML popup.

In [212]:
popup_table = [('County', "USA"),
               ('Population', '{:.0f}'.format(12345)),
               ('Area', '{:.1f} sqm'.format(678)),
               ('Assigned Trucks', '{}'.format(10)),
               ('Utilization', '{:.0%}'.format(0.33)),
               ('Relative Backlog', '{:.2%}'.format(0.02)),
               ]
html_text = get_html_table(popup_table)
html_text
# html = folium.Html(html_text, script=True)
# html

'<table style="width:100%">\n<tr><td>County&nbsp</td><td>USA</td></tr>\n<tr><td>Population&nbsp</td><td>12345</td></tr>\n<tr><td>Area&nbsp</td><td>678.0 sqm</td></tr>\n<tr><td>Assigned Trucks&nbsp</td><td>10</td></tr>\n<tr><td>Utilization&nbsp</td><td>33%</td></tr>\n<tr><td>Relative Backlog&nbsp</td><td>2.00%</td></tr></table>'

In [213]:
def create_table_plotly_popup_map():
    m = create_blank_map()
    fg = folium.FeatureGroup(name='Basic marker')
    
    popup_table = [('County', "USA"),
               ('Population', '{:.0f}'.format(12345)),
               ('Area', '{:.1f} sqm'.format(678)),
               ('Assigned Trucks', '{}'.format(10)),
               ('Utilization', '{:.0%}'.format(0.33)),
               ('Relative Backlog', '{:.2%}'.format(0.02)),
               ]
    html_text = get_html_table(popup_table)
    div_text_1 = "<div>{}</div>".format(html_text)
#     html1 = folium.Html(html_text_1, script=True)
#     iframe1 = folium.IFrame(html=html1, width=500, height=300)
#     popup = folium.Popup(iframe1, max_width=2650)
#     popup = folium.Popup(html, parse_html=True, max_width=2650)
    
    
    fig = get_plotly_pie_chart(MY_DATA)
    html2 = get_plotly_div(fig)
#     iframe1 = get_plotly_iframe(fig)
#     popup = folium.Popup(iframe2, max_width=2650)

#     iframes = iframe1 + iframe1
    html = div_text_1 + html2
    iframe = folium.IFrame(html=html, width=500, height=300)
    popup = folium.Popup(iframe, max_width=2650)
    
    # Marker with icon, tooltip and popup
    coord = boston_coord
    icon = folium.Icon(color='green',
        icon_color='white',
        icon='shopping-cart',
        angle=0, prefix='fa')
    tooltip = 'Boston'
    folium.Marker(coord,
        popup=popup,
        tooltip=tooltip,
        icon=icon
        ).add_to(fg)
    
    fg.add_to(m)
    folium.LayerControl(position='topleft').add_to(m)
    return m
create_table_plotly_popup_map()